In [62]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
from functools import reduce
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np

In [63]:
driver = webdriver.Firefox(executable_path='C:\Program Files (x86)\geckodriver.exe')
driver.get("https://geographic.org/streetview/usa/ga/index.html")

In [64]:
def page_loaded():
    main = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.TAG_NAME, "body")))

In [65]:
def hit_escape():
    try:
        page_loaded()
        driver.find_element_by_tag_name('body').send_keys(Keys.ESCAPE)
    except:
        driver.quit()

In [66]:
def load_delay():
    try:
        page_loaded()
        
    except:
        driver.quit()
        
    time.sleep(1)

In [67]:
def back_arrow():
    try:
        page_loaded()
        driver.back()
        hit_escape()
    except:
        driver.quit()

In [68]:
def click_a_link(current_link):
    try:
        page_loaded()
        current_link.click()
        hit_escape()
    except:
        driver.quit()

In [69]:
def crawl_town(link):
    
    click_a_link(link)
    
    load_delay()
    local_streets = [(element.text, county) for element in driver.find_elements_by_xpath
                         ("//li/a[contains(@href,'place=')]")]
    back_arrow()
    return (local_streets)

In [70]:
def crawl_county(link):
    
    town_streets = []
    click_a_link(link)
    
    load_delay()
    no_of_towns = len(driver.find_elements_by_xpath("//li/a[contains(@href,'.html')]"))
    for i in range(no_of_towns):
        town_streets = town_streets + crawl_town(driver.find_elements_by_xpath("//li/a[contains(@href,'.html')]")[i])
    back_arrow()
    return town_streets
        

In [71]:
county_links = driver.find_elements_by_xpath("//li/a[contains(@href,'index.html')]")
number_of_counties = len(county_links)

In [72]:
streets = []
for i in range(number_of_counties):
    county_link = driver.find_elements_by_xpath("//li/a[contains(@href,'index.html')]")[i]
    county = county_link.text
    streets = streets + crawl_county(county_link)

In [73]:
pd.DataFrame(streets, columns = ['street', 'county']).to_csv('Georgia streets.csv')

In [75]:
pd.DataFrame(streets, columns = ['street', 'county']).head().style.hide_index()

street,county
1st Street,Appling
2nd Street,Appling
A P Hunter Road,Appling
Acorn Lane,Appling
Addie Lane,Appling


In [77]:
pd.DataFrame(streets, columns = ['street', 'county']).tail()

,street,county
248735,Washington Street SE,Worth
248736,Wh Odom Road,Worth
248737,Whitehall Street,Worth
248738,Wright Road,Worth
248739,Young Avenue,Worth
